# Full Autostats Exploration (step one) 

In [ ]:
import pandas as pd 
import numpy as np 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.formula.api import glm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LassoCV
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import imblearn
import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot


import warnings
warnings.filterwarnings("ignore")

## Data Set Up

In [ ]:
## get the NBA caliber players
nba_players = pd.read_csv("NBA Caliber Players.csv")

In [ ]:
## Need to drop all start seasons before 2012 since autostats data does not go that far 
# nba_players = nba_players[nba_players['START_SEASON'] >= 2012].reset_index(drop=True)

In [ ]:
## giving those players who were not drafted but were still in the NBA a value 
nba_players['PICK_NUMBER'] = nba_players['PICK_NUMBER'].fillna('Undrafted')

In [ ]:
## autostats data 
autostats_pct = pd.read_csv('AutoStats Data for Tableau.csv')

In [ ]:
autostats_pct.columns

In [ ]:
cols = ['Player ID','Season','AutoStats Games', 'League ID','Offensive Points/Chance',
'Defensive Points/Chance',
'Offensive Transition%',
'Defensive Transition%',
'Touches/Chance',
'Points/Touch',
'Transition Touch%',
'Transition Points/Touch',
'AutoStats FG%',
'AutoStats 3P%',
'Contested Jumper FG%',
'Uncontested Jumper FG%',
'% of Jumpers Contested',
'Contested 3P%',
'Uncontested 3P%',
'% of 3s Contested',
'Off Dribble Jumper FG%',
'Catch & Shoot Jumper FG%',
'Off Dribble 3P%',
'Catch & Shoot 3P%',
'% of Shots at Rim',
'Average Influence Score',
'Expected eFG%',
'AutoStats FG% Defended',
'AutoStats 3P% Defended',
'Contested Jumper FG% Defended',
'Uncontested Jumper FG% Defended',
'% of Defended Jumpers Contested',
'Contested 3P% Defended',
'Uncontested 3P% Defended',
'% of Defended 3s Contested',
'Average Influence Score Defended',
'Expected eFG% Defended',
'Layup FG% Defended',
'Passes/Touch',
'Led to Shot %',
'Assist %',
'Secondary Assist %',
'FT Assist %',
'Offensive Adjusted Rebound %',
'Offensive Contested Rebound %',
'% of Offensive Rebounds Contested',
'Defensive Adjusted Rebound %',
'Defensive Contested Rebound %',
'% of Defensive Rebounds Contested',
'Crash & Boxout %',
'Drives/Touch',
'Player Points/Drive',
'Team Points/Drive Chance',
'Drives Defended/Defensive Chances',
'Player Points/Drive Allowed',
'Team Points/Drive Chance Allowed',
'Isos/Touch',
'Player Points/Iso',
'Team Points/Iso Chance',
'Isos Defended/Defensive Chances',
'Player Points/Iso Allowed',
'Team Points/Iso Chance Allowed',
'Postups/Touch',
'Player Points/Postup',
'Team Points/Postup Chance',
'Postups Defended/Defensive Chances',
'Player Points/Postup Allowed',
'Team Points/Postup Chance Allowed',
'BH- Screens/Touch',
'BH- Player Points/Screen',
'BH- Team Points/Chance',
'BH- Pass to Screener %',
'BH- Screener Points/Touch',
'BH Def- Ball Handler Points/Screen',
'BH Def- Screens/Chance',
'BH Def- Team Points/Chance',
'BH Def- Ball Handler Pass to Screener %',
'BH Def- Screener Points/Touch',
'SC- Screens/Chance',
'SC- Ball Handler Points/Screen',
'SC- Points/Touch',
'SC- Team Points/Chance',
'SC- Combo Points/Screen',
'SC Def- Ball Handler Points/Screen',
'SC Def- Screens/Chance',
'SC Def- Team Points/Chance',
'SC Def- Ball Handler Pass to Screener %',
'SC Def- Screener Points/Touch',
'SC Def- Combo Points/Screen',
'Closeout Touch %',
'Points/Closeout Touch- Offense',
'Team Points/Closeout Chance- Offense',
'Closeouts/Chance',
'Points/Closeout Allowed',
'Team Points/Closeout Chance Allowed',
'Full Closeout %',
'Cutter- Screens/Chance',
'Cutter- Team Points/Chance',
'Cutter- Touches/Screen',
'Cutter- Points/Touch',
'Screener- Screens/Chance',
'Screener- Team Points/Chance',
'Screener- Cutter Touches/Screen',
'Screener- Cutter Points/Touch',
'Screener- Screener Touches/Screen',
'Screener- Screener Points/Touch',
'Cutter Def- Screens/Chance',
'Cutter Def- Team Points/Chance',
'Cutter Def- Cutter Touches/Screen',
'Cutter Def- Cutter Points/Touch',
'Cutter Def- Screener Touches/Screen',
'Cutter Def- Screener Points/Touch',
'Screener Def- Screens/Chance',
'Screener Def- Team Points/Chance',
'Screener Def- Cutter Touches/Screen',
'Screener Def- Cutter Touch Points',
'Screener Def- Screener Touches/Screen',
'Screener Def- Screener Points/Touch',
'Receiver- Handoffs/Touch',
'Receiver- Points/Handoff',
'Receiver- Team Points/Chance',
'Setter- Handoffs/Touch',
'Setter- Receiver Points/Handoff',
'Setter- Team Points/Chance',
'Receiver Defender- Handoffs/Chance',
'Receiver Defender- Receiver Points/Handoff',
'Receiver Defender- Team Points/Chance',
'Setter Defender- Handoffs/Chance',
'Setter Defender- Receiver Points/Handoff',
'Setter Defender- Team Points/Chance',
'Offensive Chances',
'Defensive Chances']

In [ ]:
autostats_pct = autostats_pct[cols]

In [ ]:
## rename the player id 
autostats_pct = autostats_pct.rename({'Player ID':'PLAYER_ID'}, axis=1)

In [ ]:
## reset index
autostats_pct.reset_index(drop=True, inplace=True)

In [ ]:
## merge to get only NBA caliber players 
autostats_pct = pd.merge(nba_players, autostats_pct, how='inner', on='PLAYER_ID')

In [ ]:
autostats_pct['PLAYER_ID'].nunique()

In [ ]:
autostats_pct['Season'].unique()

In [ ]:
## grabs rows that are only from before someone started at the NBA 
## takes a while to run 
df = pd.DataFrame()
for i in range(1, len(autostats_pct)):
    if autostats_pct.loc[i, 'Season'] < autostats_pct.loc[i, 'START_SEASON']:
        df = df.append(autostats_pct.iloc[i])

In [ ]:
## make it the new dataframe 
autostats_pct = df

In [ ]:
autostats_pct.reset_index(drop=True, inplace=True)

In [ ]:
autostats_pct['PLAYER_ID'].nunique()

### Need to drop columns

columns = 1245 

columns after rank is dropped (422-1243) = 424

drop columns that are just totals = 301 + 5 (totals I need: 'PLAYER_ID', 'League ID', 'Team ID', 'Season', 'AutoStats Games')

In [ ]:
#autostats_pct = autostats_pct.drop(autostats_pct.iloc[:, 422:1243],axis = 1)

Still need PLAYER_ID, League ID, Team ID, Season, AutoStats Games

In [ ]:
#cols = list(pd.DataFrame(autostats_pct.dtypes[autostats_pct.dtypes != 'int64']).reset_index()['index'])

In [ ]:
len(cols)

In [ ]:
autostats_pct['PLAYER_ID'].nunique()

In [ ]:
#autostats_pct = pd.concat([autostats_pct[['PLAYER_ID', 'League ID', 'Team ID', 'Season', 'AutoStats Games']], autostats_pct[cols]], axis=1)

### Adding a column for drafted vs. undrafted 
`0` = NBA, non-drafted 
`1` = Drafted 

In [ ]:
## creates column for drafted and undrafted 
autostats_pct['Drafted'] = np.where(autostats_pct['PICK_NUMBER'] == 'Undrafted', 'No', 'Yes')


In [ ]:
## grab the ids, pick number and start season for later use 
pick_number = autostats_pct[['PLAYER_ID','PICK_NUMBER', 'START_SEASON']]

In [ ]:
## only want unique pairings 
pick_number = pick_number.drop_duplicates()

In [ ]:
## send to a CSV 
pick_number.to_csv("pick_num_data.csv")

In [ ]:
## dropping unneeded columns
autostats_pct = autostats_pct.drop(['START_SEASON'], axis=1)

In [ ]:
# putting the Pick Number into the bins I created

autostats_pct['PICK_NUMBER'].replace(dict.fromkeys([1,2,3,4], '1-4'), inplace=True)
autostats_pct['PICK_NUMBER'].replace(dict.fromkeys([5,6,7,8], '5-8'), inplace=True)
autostats_pct['PICK_NUMBER'].replace(dict.fromkeys([9,10,11,12], '9-12'), inplace=True)
autostats_pct['PICK_NUMBER'].replace(dict.fromkeys([13,14,15,16,17,18], '13-18'), inplace=True)

autostats_pct['PICK_NUMBER'].replace(dict.fromkeys([19,20,21,22,23,24,25,26,27,28,29,30], '19-30'), inplace=True)
autostats_pct['PICK_NUMBER'].replace(dict.fromkeys([31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60], '31-60'), inplace=True)
#autostats_pct['PICK_NUMBER'].replace(dict.fromkeys([46,47,48,49,50,51,52,53,54,55,56,57,58,59,60], '46-60'), inplace=True)

In [ ]:
## Need to convert the dashes in the data to NAs
autostats_pct = autostats_pct.replace('-', np.nan)

In [ ]:
## Look at how many NAs in each column 

na_values = list(autostats_pct.isna().sum())

## add to bottom of dataframe
autostats_pct.loc[len(autostats_pct)] = na_values

In [ ]:
## make NAs a dataframe 
na_values = pd.DataFrame(na_values)

## look at how many NAs there are 
na_values[0].sort_values().unique()

In [ ]:
## looking at which values have high nas
# na_values = pd.DataFrame(autostats_pct.isna().sum()).reset_index()
# na_values[na_values[0] > 50].sort_values(0, ascending=False)

In [ ]:
## drop the columns with more than 100 NAs
#autostats_pct = autostats_pct[autostats_pct.columns[autostats_pct.iloc[len(autostats_pct)-1] < 100]]

In [ ]:
## going to fill NAs with zeroes instead'
autostats_pct = autostats_pct.fillna(0)

## Data Aggregation 
There are several rows for each player based on season. So, I am going to build an aggregated row where depending on how many games the player played in each season will determine the percent of that data 

In [ ]:
## drop the last row
autostats_pct.drop(autostats_pct.tail(1).index,inplace=True)

In [ ]:
## creates a column to add back in later 
pick_number = autostats_pct['PICK_NUMBER']
player_id = autostats_pct['PLAYER_ID']
draft = autostats_pct['Drafted']
season = autostats_pct['Season']
#start = autostats_pct['START_SEASON']
league = autostats_pct['League ID']

In [ ]:
## sums up all AutoStats Games a player has 
a = autostats_pct.merge(pd.DataFrame(autostats_pct.groupby('PLAYER_ID') ['AutoStats Games'].sum()), on='PLAYER_ID')

## creates a percentage of each player for each season
autostats_pct['percent'] = (a['AutoStats Games_x']/a['AutoStats Games_y'])

In [ ]:
## drops columns that don't need to make floats 
autostats_pct = autostats_pct.drop(['PLAYER','PLAYER_ID','MAX_LEAGUE','PICK_NUMBER', 'Season', 'PLAYER', 'Drafted'], axis=1)
autostats_pct = autostats_pct.astype('float')

In [ ]:
## creates the columns we need to multiple the percent by
num_cols = autostats_pct.drop(['AutoStats Games'], axis=1)

In [ ]:
## adds columns back in 
autostats_pct['PICK_NUMBER'] = pick_number
autostats_pct['PLAYER_ID'] = player_id
autostats_pct['Season'] = season
autostats_pct['League ID'] = league
#autostats_pct['START_SEASON'] = start

In [ ]:
## gets the percent of each variable based on game
for i in num_cols:
    autostats_pct[i] = autostats_pct[i] * autostats_pct['percent']

In [ ]:
## add draft variable back in 
autostats_pct['Drafted'] = draft

In [ ]:
## create a rank variable that gives rank to the player and season with the most recent season being 1 
autostats_pct['rank'] = autostats_pct.groupby(['PLAYER_ID'])['Season'].rank('dense', ascending=False)


In [ ]:
l = autostats_pct[['PLAYER_ID', 'Season','rank']]
l[l['PLAYER_ID'] == 549961]

In [ ]:
## creates a variable based on the rank to multiple the values
percentage = []
for i in range(0,len(autostats_pct)):
    if autostats_pct.loc[i, 'League ID'] == 2:
        if autostats_pct.loc[i, 'rank'] == 1: 
            percentage.append(2)
        elif autostats_pct.loc[i, 'rank'] == 2: 
            percentage.append(3)
        elif autostats_pct.loc[i, 'rank'] == 3: 
            percentage.append(5)
        elif autostats_pct.loc[i, 'rank'] == 4: 
            percentage.append(5)
        elif autostats_pct.loc[i, 'rank'] == 5: 
            percentage.append(5)
        elif autostats_pct.loc[i, 'rank'] >= 6: 
            percentage.append(5)
    else: 
        percentage.append(1)

In [ ]:
## drop columns no longer needed 
num_cols.drop(['percent'], axis=1, inplace=True)
autostats_pct.drop(['Season'], axis=1, inplace=True)

In [ ]:
## make sure these are equal
print(len(percentage))
print(len(autostats_pct))

In [ ]:
## add value in 
autostats_pct['Rank_Value'] = percentage

In [ ]:
## multiplies each row by its corresponding multiplier 
for i in num_cols:
    autostats_pct[i] = autostats_pct[i] * autostats_pct['Rank_Value']

In [ ]:
## grab columns we will put in later 
cats = autostats_pct[['PLAYER_ID','PICK_NUMBER', 'Drafted', 'League ID']]
cats = cats.groupby(['PLAYER_ID']).agg({'PICK_NUMBER':'first', 'Drafted':'first', 'League ID':'first'}).reset_index()

In [ ]:
## adds up the lines a single player has 
autostats_pct = autostats_pct.groupby(['PLAYER_ID']).agg('sum').reset_index()

In [ ]:
## drops unneeded columns
autostats_pct.drop(['percent', 'rank', 'Rank_Value'], axis=1, inplace=True)

In [ ]:
## adds needed columns back in 
autostats_pct = pd.merge(autostats_pct, cats, how='inner', on='PLAYER_ID')

In [ ]:
autostats_pct = autostats_pct.drop(['League ID_x', 'League ID_y'], axis=1)

In [ ]:
autostats_pct.to_csv('autostats_data.csv')

# Exploring Options

## Dimension Reduction

In [ ]:
X_full = autostats_pct.drop(['PICK_NUMBER', 'PLAYER_ID', 'percent', 'Drafted'], axis=1)
y_full = autostats_pct['PICK_NUMBER']

In [ ]:
# Create correlation matrix
corr_matrix = X_full.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [ ]:
len(to_drop)

In [ ]:
# Drop features 
X_pct = X_full.drop(to_drop, axis=1)

## Oversampling 

Do we do PCA or SMOTE first?

In [ ]:
import pandas  as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
## Normalize the data 
X_pct=(X_pct-X_pct.mean())/(X_pct.max()-X_pct.min())

In [ ]:
y_full = y_full.astype('string')
X_pct = X_pct.astype('float')

In [ ]:
# encode labels as integers for y_train 
encoder = LabelEncoder()
encoder.fit(y_full)
y_full = encoder.transform(y_full)

In [ ]:
len(y_full)

In [ ]:
integer_mapping = {l: i for i, l in enumerate(encoder.classes_)}
integer_mapping

In [ ]:
# splitting with new data
X_train, X_test, y_train, y_test = train_test_split(X_pct, y_full, random_state=0)

In [ ]:
len(X_train)+len(X_test)

In [ ]:
len(y_train)

SMOTE

In [ ]:
# summarize distribution of y train data 
# counter = Counter(y_train)
# for k,v in counter.items():
#     per = v / len(y_train) * 100
#     print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# # plot the distribution
# pyplot.bar(counter.keys(), counter.values())
# pyplot.show()


In [ ]:
# transform the dataset of the x train to level the y trains
# oversample = SMOTE()
# X_train, y_train = oversample.fit_resample(X_train, y_train)
# # summarize distribution
# counter = Counter(y_train)
# for k,v in counter.items():
#     per = v / len(y_train) * 100
#     print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# # plot the distribution
# pyplot.bar(counter.keys(), counter.values())
# pyplot.show()

In [ ]:
# # summarize distribution of y test data 
# counter = Counter(y_test)
# for k,v in counter.items():
#     per = v / len(y_test) * 100
#     print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# # plot the distribution
# pyplot.bar(counter.keys(), counter.values())
# pyplot.show()

In [ ]:
# # transform the dataset of the x test to level the y test
# oversample = SMOTE()
# X_test, y_test = oversample.fit_resample(X_test, y_test)
# # summarize distribution
# counter = Counter(y_test)
# for k,v in counter.items():
#     per = v / len(y_test) * 100
#     print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# # plot the distribution
# pyplot.bar(counter.keys(), counter.values())
# pyplot.show()

SMOTE & Undersampling 

In [ ]:
from imblearn.combine import SMOTETomek

counter = Counter(y_train)
for k,v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.title('Train Data Counts before Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

smote_tomek = SMOTETomek(random_state=888)
X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train, y_train)

counter = Counter(y_train_resampled)
for k,v in counter.items():
    per = v / len(y_train_resampled) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.title('Train Data Counts before Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

In [ ]:
counter = Counter(y_test)
for k,v in counter.items():
    per = v / len(y_test) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.title('Test Data Counts before Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

smote_tomek = SMOTETomek(random_state=888)
X_test_resampled, y_test_resampled = smote_tomek.fit_resample(X_test, y_test)

counter = Counter(y_test_resampled)
for k,v in counter.items():
    per = v / len(y_test_resampled) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.title('Test Data Counts After Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

### Model for drafted vs. non-drafted NBA

First, Need to create a variable for drafted vs. non-drafted

`0` = NBA, non-drafted 
`1` = Drafted 

Model Set up

In [ ]:
# X_rank = autostats_pct.drop(['PICK_NUMBER', 'PLAYER_ID'], axis=1)
# Y_pct = autostats_pct['PICK_NUMBER']

In [ ]:
## Fill NAs with mean 
#X_rank = X_rank.fillna(X_rank.mean())

In [ ]:
## Don't need this unless we are not doing the aggregated data 
# X_rank = X_rank.fillna(0)

Don't need this since we already dropped it

#### PCA 

need to normalize first

In [ ]:
pca_all = PCA(n_components=len(X_train_resampled.columns))
pca_all.fit(X_train_resampled)

In [ ]:
plt.grid()
plt.plot(np.cumsum(pca_all.explained_variance_ratio_ * 100))
plt.xlabel('Number of components')
plt.xlim([0, 200])
plt.ylabel('Explained variance')

In [ ]:
# train and test sets
pca_100 = PCA(n_components=100)
pca_100.fit(X_train_resampled)
X_train_reduced = pca_100.transform(X_train_resampled)
X_test_reduced = pca_100.transform(X_test_resampled)

# verify shape after PCA
print("Shape:", X_train_resampled.shape)
print("Test images shape: ", X_test_resampled.shape)

# get exact variability retained
print("\nVar retained (%):", 
      np.sum(pca_100.explained_variance_ratio_ * 100))

### Random Forest 

`0` : '1-4'

`1`: '13-18'

`2`: '19-30'

`3`: '31-60'

`4`: '5-8'

`5`: 9-12

In [ ]:
# Instantiate and fit the RandomForestClassifier
forest = RandomForestClassifier()
forest.fit(X_train_reduced, y_train_resampled)

In [ ]:
# Use the forest's predict method on the test data
predictions = forest.predict(X_test_reduced)

# View accuracy score
accuracy_score(y_test_reduced, predictions)

# Evaluation metrics
print(classification_report(y_test_resampled, predictions))

In [ ]:
confusion_matrix(y_test_resampled, predictions)

***Problem:*** it is only guessing one category

### Penalized-SVM 
*** Don't Actually need this anymore since we oversampled ***

In [ ]:
# load library
from sklearn.svm import SVC
import sklearn.metrics

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(class_weight='balanced', probability=True)

svc_model.fit(X_train_reduced, y_train)

svc_predict = svc_model.predict(X_test_reduced)# check performance
#print('ROCAUC score:',roc_auc_score(y_test, svc_predict))
print('Accuracy score:',accuracy_score(y_test, svc_predict))
#print('F1 score:',f1_score(y_test, svc_predict))

In [ ]:
print(classification_report(y_test, svc_predict))

In [ ]:
print(confusion_matrix(y_test, svc_predict))

### Random Forest with Balanced Weights
*** Don't Actually need this anymore since we oversampled ***

In [ ]:
# creating model and fitting it 
balanced_rf = RandomForestClassifier(class_weight='balanced', random_state=0, n_jobs=-1).fit(X_train_resampled, y_train)

# Baseline model prediction
y_test_pred_balanced = balanced_rf.predict(X_test_reduced)

# Evaluation metrics
print(classification_report(y_test, y_test_pred_balanced))

Support Vector Machines do a better job of helping with imbalanced data even thought the random forest has a higher accuracy but that is just because it is only guessing two categories

## Artifical Neural Network

- '1-4': `0`
- '5-8': `4`
- '9-12': `5`
- '13-18': `1`
- '19-30': `2`
- '31-60': `3`
- 'Other- NBA': `6`

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import keras

In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = np_utils.to_categorical(y_train_resampled)

In [ ]:
list(encoder.classes_)

In [ ]:
## allows you to see which categorical value it is 
# list(encoder.inverse_transform(encoded_Y_train));

In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_test = np_utils.to_categorical(y_test_resampled)

In [ ]:
list(encoder.classes_)

#### One Way to do this

Need to create class weights since it is predicting only one class the whole time 

`0` : '1-4'

`1`: '13-18'

`2`: '19-30'

`3`: '31-60'

`4`: '5-8'

`5`: 9-12

*** Don't really need class weights anymore either ***

In [ ]:
# build a model
model = Sequential()
model.add(Dense(50, input_shape=(X_train_reduced.shape[1],), activation='relu')) # input shape is (features,)
model.add(Dense(25, activation='softmax'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(7, activation='softmax'))
model.summary()

# compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', # this is different instead of binary_crossentropy (for regular classification)
              metrics=['accuracy'])

In [ ]:
# early stopping callback
# This callback will stop the training when there is no improvement in  
# the validation loss for 10 consecutive epochs.  
es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                   mode='min',
                                   patience=10, 
                                   restore_best_weights=True) 

# now we just update our model fit call
history =  model.fit(X_train_reduced,
                    dummy_y_train,
                    callbacks=[es],
                    epochs=5000, 
                    # class_weight = class_weights,
                    batch_size=10,
                    shuffle=True,
                    validation_split=0.2,
                    verbose=1)

In [ ]:
history_dict = history.history

# learning curve
# accuracy
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

# loss
loss = history_dict['loss']
val_loss = history_dict['val_loss']

# range of X (no. of epochs)
epochs = range(1, len(acc) + 1)

# plot
# "r" is for "solid red line"
plt.plot(epochs, acc, 'r', label='Training accuracy')
# b is for "solid blue line"
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

preds = model.predict(X_train_reduced) # see how the model did!
print(preds[4]) # i'm spreading that prediction across three nodes and they sum to 1
print(np.sum(preds[0])) # sum it up! Should be 1

This is a problem

In [ ]:
predictions = preds.argmax(axis=1)

In [ ]:
cm = confusion_matrix(y_train_resampled, predictions)

`0` : '1-4'

`1`: '13-18'

`2`: '19-30'

`3`: '31-60'

`4`: '5-8'

`5`: 9-12

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other']); ax.yaxis.set_ticklabels(['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other']);

Looking at the confusion matrix we can see that for most of the labels we are good in the 40% while for 19-30 there are no predictions but seem to be close to the 31-60

In [ ]:
predictions[14]

In [ ]:
preds[14]

#### Another Way 

In [ ]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense
# Neural network
model2 = Sequential()
model2.add(Dense(16, input_dim=100, activation='relu'))
model2.add(Dense(12, activation='relu'))
model2.add(Dense(7, activation='softmax'))

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history2 = model2.fit(X_train_reduced, dummy_y_train, epochs=100, batch_size=64);

In [ ]:
y_pred = model2.predict(X_test_reduced)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(dummy_y_test)):
    test.append(np.argmax(dummy_y_test[i]))

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

In [ ]:
history2 = model2.fit(X_train_reduced, dummy_y_train, validation_data = (X_test_reduced,dummy_y_test), epochs=100,batch_size=64);

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
cm = confusion_matrix(y_train_resampled, predictions)

Maybe try bootstrapping???